In [1]:
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
from numba import jit
from tqdm import tqdm
import itertools
from itertools import combinations, permutations
import time

In [2]:
#data_A = np.loadtxt('../data/TSPA.csv', delimiter=';').astype(np.int64)
#data_B = np.loadtxt('../data/TSPB.csv', delimiter=';').astype(np.int64)
data_C = np.loadtxt('../data/TSPC.csv', delimiter=';').astype(np.int64)
data_D = np.loadtxt('../data/TSPD.csv', delimiter=';').astype(np.int64)

In [3]:
def create_cost_matrix(data):
    x = data[:, :1]
    y = data[:, 1:2]
    cost = data[:, 2:3]
    return (((x - x.reshape(1, -1))**2 + (y - y.reshape(1, -1))**2) ** (1/2) + cost.reshape(1, -1)).round().astype(np.int64)

def create_dist_matrix(data):
    x = data[:, :1]
    y = data[:, 1:2]
    #cost = data[:, 2:3]
    return (((x - x.reshape(1, -1))**2 + (y - y.reshape(1, -1))**2) ** (1/2)).round().astype(np.int64)

In [4]:
cost_matrix_C = create_cost_matrix(data_C)

In [5]:
cost_matrix_D = create_cost_matrix(data_D)

In [6]:
#dist_matrix_A = create_dist_matrix(data_A)
#dist_matrix_B = create_dist_matrix(data_B)
dist_matrix_C = create_dist_matrix(data_C)
dist_matrix_D = create_dist_matrix(data_D)

In [7]:
def plot(data, solution):
    data_ordered = np.array([data[i] for i in solution])
    all_data = np.array([data[i] for i in range(200)])

    plt.figure(figsize=(10, 10), dpi=80)

    plt.scatter(data_ordered[:,0], data_ordered[:,1], s=data_ordered[:,2]/data_ordered[:,2].max()*200, c='b')
    plt.scatter(all_data[:,0], all_data[:,1], s=all_data[:,2]/all_data[:,2].max()*200, c='b')
    plt.plot(data_ordered[:,0], data_ordered[:,1], 'y-')
    plt.plot([data_ordered[0,0], data_ordered[-1,0]], [data_ordered[0,1], data_ordered[-1,1]], 'y-')
    plt.show()

In [8]:
def calculate_performance(cycle, cost_matrix):
    total_sum = 0
    for i in range(len(cycle)-1):
        total_sum += cost_matrix[cycle[i], cycle[i+1]]
    total_sum += cost_matrix[cycle[-1], cycle[0]]
    return total_sum

In [10]:
def random_solution(cost_matrix, limit=100):
    random_solution_list = list(range(0,len(cost_matrix)))
    random.shuffle(random_solution_list)
    return np.array(random_solution_list)[:limit]

solution = random_solution(cost_matrix_C, 100)
print(solution)

[  9  74 149  72  38 141 193 108 195  13 110  18  60  62  40  56  15  93
 194  47  17  53 107 124  64 192  46  42   2   4 142 140 133  51   1 154
  25  34 198  11   6 167 196 161 113 114  68 134 120 116 155  26  85  50
  22 191 180 138  23 165  14  95 117  79 178  31 105 152 106 151 125  35
  49  88  61 186 123   0  44 143 112  77 136 182 184  63  19 128 162 197
  67 104 132 122 158  78  54  66 171 121]


In [11]:
def greedy_cycle(cost_matrix, current_id, limit=100):
    all_ids = set(list(range(0,len(cost_matrix))))
    all_ids.remove(current_id)
    solution = [current_id]
    
    for _ in range(1):
        min_val = 99999
        min_id = -1
        for next_id in all_ids:
            if cost_matrix[current_id][next_id] < min_val:
                min_val = cost_matrix[current_id][next_id]
                min_id = next_id
        solution.append(min_id)
        all_ids.remove(min_id)
        current_id = min_id
    
    while len(solution) < limit:
        min_delta = 99999
        min_id = -1
        insert_id = -1
        for i in range(len(solution)-1):
            for next_id in all_ids:
                delta = cost_matrix[solution[i]][next_id] + cost_matrix[next_id][solution[i+1]] - cost_matrix[solution[i]][solution[i+1]]
                if delta < min_delta:
                    min_delta = delta
                    min_id = next_id
                    insert_id = i
        for next_id in all_ids:
            delta = cost_matrix[solution[-1]][next_id] + cost_matrix[next_id][solution[0]] - cost_matrix[solution[-1]][solution[0]]
            if delta < min_delta:
                min_delta = delta
                min_id = next_id
                insert_id = i
        solution.insert(insert_id+1, min_id)
        all_ids.remove(min_id)

    return np.array(solution)

In [12]:
# swap node in cycle and unused node
def inter_exchange_delta(cycle, cost_matrix, used_cycle_id, unused_node):
    if used_cycle_id == len(cycle)-1:
        return cost_matrix[cycle[used_cycle_id-1], unused_node] + cost_matrix[unused_node, cycle[0]] - cost_matrix[cycle[used_cycle_id-1], cycle[used_cycle_id]] - cost_matrix[cycle[used_cycle_id], cycle[0]]
    return cost_matrix[cycle[used_cycle_id-1], unused_node] + cost_matrix[unused_node, cycle[used_cycle_id+1]] - cost_matrix[cycle[used_cycle_id-1], cycle[used_cycle_id]] - cost_matrix[cycle[used_cycle_id], cycle[used_cycle_id+1]]

In [13]:
def intra_two_nodes_exchange_delta(cycle, cost_matrix, node1, node2, pri = False):
    # print(node1, node2)
    if node1 == len(cycle)-1:
        node1plus = 0
    else:
        node1plus = node1 + 1
    
    if node2 == len(cycle)-1:
        node2plus = 0
    else:
        node2plus = node2 + 1

    if abs(node1-node2) == 1:
        if node1 > node2:
            node1, node2 = node2, node1
            node1plus, node2plus = node2plus, node1plus
        return cost_matrix[cycle[node1-1], cycle[node2]] + cost_matrix[cycle[node2], cycle[node1]] + cost_matrix[cycle[node1], cycle[node2plus]] - cost_matrix[cycle[node1-1], cycle[node1]] - cost_matrix[cycle[node1], cycle[node2]] - cost_matrix[cycle[node2], cycle[node2plus]]
    if abs(node1-node2) == len(cycle)-1:
        if node1 < node2:
            node1, node2 = node2, node1
            node1plus, node2plus = node2plus, node1plus
        return cost_matrix[cycle[node1-1], cycle[node2]] + cost_matrix[cycle[node2], cycle[node1]] + cost_matrix[cycle[node1], cycle[node2plus]] - cost_matrix[cycle[node1-1], cycle[node1]] - cost_matrix[cycle[node1], cycle[node2]] - cost_matrix[cycle[node2], cycle[node2plus]]

    return cost_matrix[cycle[node1-1], cycle[node2]] + cost_matrix[cycle[node2], cycle[node1plus]] + cost_matrix[cycle[node2-1], cycle[node1]] + cost_matrix[cycle[node1], cycle[node2plus]] - cost_matrix[cycle[node1-1], cycle[node1]] - cost_matrix[cycle[node1], cycle[node1plus]] - cost_matrix[cycle[node2-1], cycle[node2]] - cost_matrix[cycle[node2], cycle[node2plus]]

In [14]:
def intra_two_edges_exchange_delta(cycle, dist_matrix, node1, node2):
    if node1 == len(cycle)-1 or node2 == len(cycle)-1 or abs(node1-node2) < 2:
        return 99999
    #print(cost_matrix[cycle[node1], cycle[node2]], cost_matrix[cycle[node1+1], cycle[node2+1]], cost_matrix[cycle[node1], cycle[node1+1]], cost_matrix[cycle[node2], cycle[node2+1]])
    return dist_matrix[cycle[node1], cycle[node2]] + dist_matrix[cycle[node1+1], cycle[node2+1]] - dist_matrix[cycle[node1], cycle[node1+1]] - dist_matrix[cycle[node2], cycle[node2+1]]


In [15]:
def steepest_local_search(cost_matrix, dist_matrix, nodes_exchange = True, initial_greedy = False, starting_node = None):
    if initial_greedy:
        if starting_node:
            cycle = greedy_cycle(cost_matrix, starting_node)
        else:
            cycle = greedy_cycle(cost_matrix, random.randint(0, len(cost_matrix)-1))
    else:
        cycle = random_solution(cost_matrix)
    unused_nodes = set(list(range(0,len(cost_matrix))))
    for node in cycle:
        unused_nodes.remove(node)

    all_combinations = list(combinations(list(range(0, 100)), 2))

    while(True):
        best_delta, swap_node_a, swap_node_b, best_type = 0, None, None, None
        for used_cycle_id in range(len(cycle)):
            for unused_node in unused_nodes:
                delta = inter_exchange_delta(cycle, cost_matrix, used_cycle_id, unused_node)
                if delta < best_delta:
                    best_delta, swap_node_a, swap_node_b, best_type = delta, used_cycle_id, unused_node, 'inter_exchange'
        if nodes_exchange:
            for x, y in all_combinations:
                delta = intra_two_nodes_exchange_delta(cycle, cost_matrix, x, y)
                if delta < best_delta:
                    best_delta, swap_node_a, swap_node_b, best_type = delta, x, y, 'intra_two_nodes_exchange'
        else:
            for x, y in all_combinations:
                delta = intra_two_edges_exchange_delta(cycle, dist_matrix, x, y)
                if delta < best_delta:
                    best_delta, swap_node_a, swap_node_b, best_type = delta, x, y, 'intra_two_edges_exchange'

        if best_type is not None:
            if best_type == 'inter_exchange':
                unused_nodes.add(cycle[swap_node_a])
                unused_nodes.remove(swap_node_b)
                cycle[swap_node_a] = swap_node_b
            elif best_type == 'intra_two_nodes_exchange':
                cycle[swap_node_a], cycle[swap_node_b] = cycle[swap_node_b], cycle[swap_node_a]
            elif best_type == 'intra_two_edges_exchange':
                cycle[swap_node_a+1], cycle[swap_node_b] = cycle[swap_node_b], cycle[swap_node_a+1]
                cycle[swap_node_a+2:swap_node_b] = cycle[swap_node_a+2:swap_node_b][::-1]
        else:
            break
    return cycle

In [16]:
def steepest_local_search_edge_exchange_improving_moves(cost_matrix, dist_matrix):
    cycle = random_solution(cost_matrix)
    unused_nodes = set(list(range(0,len(cost_matrix))))
    used_nodes = set()
    for node in cycle:
        unused_nodes.remove(node)
        used_nodes.add(node)

    all_combinations = list(combinations(list(range(0, 100)), 2))

    # Initiate LM – a list of moves that bring improvement ordered from the best to the worst
    LM = []
    for used_cycle_id in range(len(cycle)):
        for unused_node in unused_nodes:
            delta = inter_exchange_delta(cycle, cost_matrix, used_cycle_id, unused_node)
            if delta < 0:
                LM.append((delta, 'inter_exchange', used_cycle_id, cycle[used_cycle_id], unused_node, 0, 0, 0))
    for x, y in all_combinations:
        delta = intra_two_edges_exchange_delta(cycle, dist_matrix, x, y)
        if delta < 0:
            LM.append((delta, 'intra_two_edges_exchange', x, y, cycle[x], cycle[x+1], cycle[y], cycle[y+1]))
    LM.sort(key=lambda x: x[0])
    LM = np.array(LM, dtype=object)

    # until no move has been found after checking the whole list LM
    any_applicable = True
    to_remove = []
    while(any_applicable):

        new_moves_intra, new_moves_inter, new_moves_inter_from_intra, new_moves_intra_from_inter = None, None, None, None
        any_applicable = False
        to_remove = []

        # for moves m from LM starting from the best until a applicable move is found
        for k in range(len(LM)):
            move = LM[k]
            if move[1] == 'intra_two_edges_exchange':    
                node1, node1plus, node2, node2plus = move[4], move[5], move[6], move[7]
                node1_next, node1_prev, node2_next, node2_prev = False, False, False, False
                idx1, idx2 = move[2], move[3]
                if idx1 > idx2:
                    node1, node2 = node2, node1
                    node1plus, node2plus = node2plus, node1plus
                    idx1, idx2 = idx2, idx1

                if cycle[idx1] == node1 and cycle[idx1+1] == node1plus:
                    node1_next = True
                elif cycle[idx1] == node1plus and cycle[idx1+1] == node1:
                    node1_prev = True
                
                if cycle[idx2] == node2 and cycle[idx2+1] == node2plus:
                    node2_next = True
                elif cycle[idx2] == node2plus and cycle[idx2+1] == node2:
                    node2_prev = True

                # TODO: repair
                if node1_next and node2_next:
                    # apply move and remove from LM
                    cycle[idx1+1], cycle[idx2] = cycle[idx2], cycle[idx1+1]
                    cycle[idx1+2:idx2] = cycle[idx1+2:idx2][::-1]                    
                    to_remove.append(k)
                    any_applicable = True
                    new_moves_intra = (idx1, idx2)
                    new_moves_inter_from_intra = (idx1, idx1+1, idx2, idx2+1)
                    break
                elif node1_prev and node2_prev:
                    # apply move and remove from LM
                    cycle[idx1], cycle[idx2-1] = cycle[idx2-1], cycle[idx1]
                    cycle[idx1+1:idx2-1] = cycle[idx1+1:idx2-1][::-1]
                    to_remove.append(k)
                    any_applicable = True
                    new_moves_intra = (idx1, idx2)
                    new_moves_inter_from_intra = (idx1, idx1+1, idx2, idx2+1)
                    break
                if (node1_next and node2_prev) or (node1_prev and node2_next):
                    continue
                to_remove.append(k)

            if move[1] == 'inter_exchange':
                used_cycle_id, used_node, unused_node = move[2], move[3], move[4]
                # check if move is applicable
                if cycle[used_cycle_id] == used_node and used_node in used_nodes and unused_node in unused_nodes:
                    unused_nodes.add(used_node)
                    unused_nodes.remove(unused_node)
                    used_nodes.add(unused_node)
                    used_nodes.remove(used_node)
                    cycle[used_cycle_id] = unused_node
                    any_applicable = True
                    new_moves_inter = (used_cycle_id, used_node)
                    new_moves_intra_from_inter = (used_cycle_id)
                    to_remove.append(k)
                    break
                to_remove.append(k)

        LM = np.delete(LM, to_remove, axis=0)

        # INTER
        newLM = []
        if new_moves_inter:
            used_cycle_id, old_used_node = new_moves_inter
            LM = LM[(LM[:,1] != 'inter_exchange') | (LM[:,2] != used_cycle_id)]
            LM = LM[(LM[:,1] != 'inter_exchange') | (LM[:,2] != used_cycle_id-1)]
            LM = LM[(LM[:,1] != 'inter_exchange') | (LM[:,2] != ((used_cycle_id + 1)%len(cycle)))]
            LM = LM[(LM[:,1] != 'inter_exchange') | (LM[:,4] != old_used_node)]

            for unus in unused_nodes:
                delta = inter_exchange_delta(cycle, cost_matrix, used_cycle_id, unus)
                if delta < 0:
                    newLM.append((delta, 'inter_exchange', used_cycle_id, cycle[used_cycle_id], unus, 0, 0, 0))

                idx = used_cycle_id - 1
                delta = inter_exchange_delta(cycle, cost_matrix, idx, unus)
                if delta < 0:
                    newLM.append((delta, 'inter_exchange', idx, cycle[idx], unus, 0, 0, 0))

                idx = (used_cycle_id + 1)%len(cycle)
                delta = inter_exchange_delta(cycle, cost_matrix, idx, unus)
                if delta < 0:
                    newLM.append((delta, 'inter_exchange', idx, cycle[idx], unus, 0, 0, 0))
                    
            for i in range(len(cycle)):
                delta = inter_exchange_delta(cycle, cost_matrix, i, old_used_node)
                if delta < 0:
                    newLM.append((delta, 'inter_exchange', i, cycle[i], unus, 0, 0, 0))


        # INTRA
        if new_moves_intra:
            x, y = new_moves_intra
            LM = LM[(LM[:,1] != 'intra_two_edges_exchange') | (LM[:,2] != used_cycle_id)]
            if x > y:
                x, y = y, x
            all_combinations = list(itertools.product(list(range(len(cycle))), list(range(x, y+2))))
            for z, b in all_combinations:
                delta = intra_two_edges_exchange_delta(cycle, dist_matrix, z, b)
                if delta < 0:
                    newLM.append((delta, 'intra_two_edges_exchange', z, b, cycle[z], cycle[z+1], cycle[b], cycle[b+1]))
        
        # NEW FROM OTHER
        if new_moves_intra_from_inter:
            used_cycle_id = new_moves_intra_from_inter
            if used_cycle_id < 0:
                used_cycle_id = 0
            all_combinations = []

            all_combinations = list(itertools.product(list(range(len(cycle))), list(range(used_cycle_id-1, (used_cycle_id+3)%len(cycle)))))
            for z, b in all_combinations:
                delta = intra_two_edges_exchange_delta(cycle, dist_matrix, z, b)
                if delta < 0:
                    newLM.append((delta, 'intra_two_edges_exchange', z, b, cycle[z], cycle[z+1], cycle[b], cycle[b+1]))

        if new_moves_inter_from_intra:
            for used_cycle_id in new_moves_inter_from_intra:
                LM = LM[(LM[:,1] != 'inter_exchange') | (LM[:,2] != used_cycle_id)]
                for unus in unused_nodes:
                    delta = inter_exchange_delta(cycle, cost_matrix, used_cycle_id, unus)
                    if delta < 0:
                        newLM.append((delta, 'inter_exchange', used_cycle_id, cycle[used_cycle_id], unus, 0, 0, 0))
        
        newLM = np.array(newLM, dtype=object)
        if len(newLM) > 0:
            LM = np.append(LM, newLM, axis=0)
        LM = LM[LM[:, 0].argsort()]
    
    return cycle

In [49]:
# Optional profiling
%load_ext line_profiler

In [ ]:
%lprun -f steepest_local_search_edge_exchange_improving_moves steepest_local_search_edge_exchange_improving_moves(cost_matrix_C, dist_matrix_C)

Tests

In [17]:
def present_results(min, max, average, time):
    print("{: >10} {: >10} {: >10} {: >10}".format('MIN', 'MAX', 'AVG', 'TIME'))
    print("{: >10} {: >10} {: >10} {: >10}".format(min, max, average, time))
    print()

In [18]:
# local_search(cost_matrix, inter = True, initial_greedy = False, starting_node = None)
def test_solution(cost_matrix, dist_matrix, improving_moves):
    start = time.time()
    costs = []
    for i in tqdm(range(len(cost_matrix))):
        if improving_moves:
            cycle = steepest_local_search_edge_exchange_improving_moves(cost_matrix, dist_matrix)
        else:
            cycle = steepest_local_search(cost_matrix, dist_matrix, nodes_exchange = False, initial_greedy = False)
        total_cost = calculate_performance(cycle, cost_matrix)
        costs.append(total_cost)
    costs = np.array(costs)
    return costs.min(), costs.max(), costs.mean(), time.time() - start

local search | neighborhood | starting solutions

In [24]:
print('DATASET C - STEEPEST | EDGES | RANDOM - WITHOUT IMPROVING MOVES')
present_results(*test_solution(cost_matrix_C, dist_matrix_C, False))

DATASET C - STEEPEST | EDGES | RANDOM - WITHOUT IMPROVING MOVES


100%|██████████| 200/200 [07:50<00:00,  2.35s/it]

       MIN        MAX        AVG       TIME
     49109      55709  51777.445 470.04248428344727



In [23]:
print('DATASET D - STEEPEST | EDGES | RANDOM - WITHOUT IMPROVING MOVES')
present_results(*test_solution(cost_matrix_D, dist_matrix_D, False))

DATASET D - STEEPEST | EDGES | RANDOM - WITHOUT IMPROVING MOVES


100%|██████████| 200/200 [07:57<00:00,  2.39s/it]

       MIN        MAX        AVG       TIME
     45676      52036   48497.86 477.2460265159607



In [24]:
print('DATASET C - STEEPEST | EDGES | RANDOM - WITH IMPROVING MOVES')
present_results(*test_solution(cost_matrix_C, dist_matrix_C, True))

DATASET C - STEEPEST | EDGES | RANDOM - WITH IMPROVING MOVES


100%|██████████| 200/200 [05:35<00:00,  1.68s/it]

       MIN        MAX        AVG       TIME
     49920      56447   52410.28 335.9229564666748



In [25]:
print('DATASET D - STEEPEST | EDGES | RANDOM - WITH IMPROVING MOVES')
present_results(*test_solution(cost_matrix_D, dist_matrix_D, True))

DATASET D - STEEPEST | EDGES | RANDOM - WITH IMPROVING MOVES


100%|██████████| 200/200 [05:43<00:00,  1.72s/it]

       MIN        MAX        AVG       TIME
     46839      54113  49456.445 343.73134899139404

